In [4]:
from dotenv import load_dotenv
import os
import pickle


In [2]:
load_dotenv()

True

In [5]:
path = '../data/Constitución Española/chunks'
file_path = os.path.join(path, 'documents_spanisharticlesplitter.pkl')
with open(file_path, "rb") as file:
    texts = pickle.load(file)

create vector store

In [7]:
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma
import os

embedding_function = AzureOpenAIEmbeddings(
    azure_deployment=os.environ["AZURE_ADA2_DEPLOYMENT"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
)

# load docs into Chroma
vector_db = Chroma.from_documents(texts, embedding_function, persist_directory='../vector_stores/constitucion_española')

# Helpful to force a save
vector_db.persist()

# get db connection
vector_db_connection = Chroma(persist_directory='../vector_stores/constitucion_española', embedding_function=embedding_function)

/home/alvaro/tfm/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [8]:
retriever = vector_db_connection.as_retriever(search_kwargs={"k": 2})

In [9]:
from langchain_openai import AzureChatOpenAI
from langchain.chains import RetrievalQA

# create a retrieval qa chain using llm
llm = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_GPT3TURBO_DEPLOYMENT"],
)
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 verbose=True)

In [30]:
question = "¿Cuál es el objetivo de la educación según los principios constitucionales?"
answer = qa.invoke(question)
context = retriever.get_relevant_documents(question)
context_text = '\n\n'.join([c.page_content for c in context])
message = f"""Question: {question}"

Answer: {answer['result']}")

Context: 

{context_text}"""



> Entering new RetrievalQA chain...

> Finished chain.


In [31]:
print(message)

Question: ¿Cuál es el objetivo de la educación según los principios constitucionales?"

Answer: Según los principios constitucionales, el objetivo de la educación es el pleno desarrollo de la personalidad humana en el respeto a los principios democráticos de convivencia y a los derechos y libertades fundamentales.")

Context: 

1. Todos tienen el derecho a la educación. Se reconoce la libertad de enseñanza.
2. La educación tendrá por objeto el pleno desarrollo de la personalidad humana en el respeto a los principios democráticos de convivencia y a los derechos y libertades fundamentales.
3. Los poderes públicos garantizan el derecho que asiste a los padres para que sus hijos reciban la formación religiosa y moral que esté de acuerdo con sus propias convicciones.
4. La enseñanza básica es obligatoria y gratuita.
5. Los poderes públicos garantizan el derecho de todos a la educación, mediante una programación general de la enseñanza, con participación efectiva de todos los sectores afecta